In [65]:
import datetime as dt
import pandas as pd
from fredapi import Fred
fred = Fred(api_key='9052ad007adeecfed40214e9552318d5')

In [66]:
country_id_map = {
    'Australia': 'IRLTLT01AUM156N',
    'Belgium': 'IRLTLT01BEM156N',
    'Canada': 'IRLTLT01CAM156N',
    'Denmark': 'IRLTLT01DKM156N',
    'France': 'IRLTLT01FRM156N',
    'Germany': 'IRLTLT01DEM156N',
    'Greece': 'IRLTLT01GRM156N',
    'India': 'INDIRLTLT01STM',
    'Ireland': 'IRLTLT01IEM156N',
    'Italy': 'IRLTLT01ITM156N',
    'Japan': 'IRLTLT01JPM156N',
    'Korea, Rep.': 'IRLTLT01KRM156N',
    'Netherlands': 'IRLTLT01NLM156N',
    'New Zealand': 'IRLTLT01NZM156N',
    'Norway': 'IRLTLT01NOM156N',
    'Poland': 'IRLTLT01PLM156N',
    'Portugal': 'IRLTLT01PLM156N',
    'South Africa': 'IRLTLT01ZAM156N',
    'Spain': 'IRLTLT01ESM156N',
    'Sweden': 'IRLTLT01SEM156N',
    'United Kingdom': 'IRLTLT01GBM156N',
    'United States': 'IRLTLT01USM156N'

}

In [67]:
%cd ..
%cd data

credit_df = pd.read_csv('credit_ratings.csv')
credit_df.head()

/Users/jamiewong/Documents/esg-financial-assistant
/Users/jamiewong/Documents/esg-financial-assistant/data


,Country Name,Credit Rating
0,Australia,100
1,Belgium,87
2,Canada,100
3,Denmark,100
4,France,88


In [68]:
esg_df = pd.read_csv('country_esg_scores.csv')
esg_df.head()

,Country Name,E Score,S Score,G Score,ESG Score
0,Sweden,0.851642,0.845986,0.772606,2.470234
1,Norway,0.828819,0.799264,0.817535,2.445618
2,Denmark,0.750027,0.803905,0.846567,2.400499
3,New Zealand,0.594863,0.854180,0.848597,2.297640
4,Germany,0.767347,0.772925,0.736296,2.276569


In [69]:
merged_df = pd.merge(credit_df, esg_df, on = 'Country Name', how = 'left')
merged_df.head()

,Country Name,Credit Rating,E Score,S Score,G Score,ESG Score
0,Australia,100,0.547563,0.866146,0.821755,2.235464
1,Belgium,87,0.662349,0.773847,0.647842,2.084038
2,Canada,100,0.484865,0.779746,0.752887,2.017498
3,Denmark,100,0.750027,0.803905,0.846567,2.400499
4,France,88,0.797240,0.764189,0.625450,2.186880


In [70]:
yields_df = pd.DataFrame()
obs_yr = dt.datetime.now().year-1
obs_mo = dt.datetime.now().month-1
for country in country_id_map:
    yields_df[country] = fred.get_series(country_id_map[country], observation_start=f'{obs_yr}-{obs_mo}-01')

In [71]:
yields_df

,Australia,Belgium,Canada,Denmark,France,Germany,Greece,India,Ireland,Italy,...,Netherlands,New Zealand,Norway,Poland,Portugal,South Africa,Spain,Sweden,United Kingdom,United States
2024-06-01,4.242,3.12,3.391500,2.518,3.15,2.484500,3.65,7.02,2.9287,3.942,...,2.822,4.64,3.537450,5.73,5.73,11.678947,3.356050,2.25761,4.1644,4.31
2024-07-01,4.326,3.06,3.407727,2.448,3.14,2.458261,3.50,7.01,2.8587,3.830,...,2.765,4.51,3.544522,5.65,5.65,11.021739,3.280391,2.12817,4.1386,4.25
2024-08-01,3.976,2.83,3.075714,2.199,2.94,2.207727,3.31,6.91,2.6370,3.682,...,2.531,4.22,3.296455,5.28,5.28,10.700000,3.071045,1.93209,3.9444,3.87
2024-09-01,3.922,2.79,2.944737,2.114,2.90,2.167143,3.19,6.83,2.5457,3.565,...,2.469,4.19,3.298762,5.31,5.31,10.301000,2.995667,1.92729,3.9053,3.72
2024-10-01,4.267,2.84,3.186364,2.104,2.99,2.229565,3.16,6.79,2.5888,3.501,...,2.533,4.39,3.541391,5.60,5.60,10.464348,2.967609,2.04304,4.1993,4.10
2024-11-01,4.544,2.92,3.278500,2.069,3.09,2.305714,3.19,6.82,2.6262,3.569,...,2.567,4.62,3.668714,5.66,5.66,10.368095,3.055619,2.06743,4.4164,4.36
2024-12-01,4.313,2.80,3.161500,1.912,3.01,2.178889,3.05,6.78,2.4906,3.321,...,2.447,4.49,3.598889,5.73,5.73,10.252105,2.890650,2.10150,4.4345,4.39
2025-01-01,4.481,3.10,3.288636,2.213,3.32,2.483636,3.35,6.76,2.7890,3.676,...,2.732,4.65,3.884409,5.94,5.94,10.419545,3.173182,2.32138,4.6627,4.63
2025-02-01,4.423,3.01,2.935000,2.166,3.15,2.405500,3.29,6.73,2.7164,3.575,...,2.629,4.62,3.880150,5.83,5.83,10.497000,3.099850,2.25515,4.5063,4.45
2025-03-01,4.421,3.32,3.037143,2.454,3.43,2.741429,3.60,6.68,3.0487,3.887,...,2.936,4.63,4.017476,5.86,5.86,10.998000,3.389190,2.60248,4.6448,4.28


In [72]:
yields_df =  yields_df.transpose()
yields_df = yields_df.reset_index(names = 'Country Name')
yields_df

,Country Name,2024-06-01 00:00:00,2024-07-01 00:00:00,2024-08-01 00:00:00,2024-09-01 00:00:00,2024-10-01 00:00:00,2024-11-01 00:00:00,2024-12-01 00:00:00,2025-01-01 00:00:00,2025-02-01 00:00:00,2025-03-01 00:00:00,2025-04-01 00:00:00,2025-05-01 00:00:00,2025-06-01 00:00:00
0,Australia,4.242000,4.326000,3.976000,3.922000,4.267000,4.544000,4.313000,4.481000,4.42300,4.421000,4.267000,4.350000,4.220000
1,Belgium,3.120000,3.060000,2.830000,2.790000,2.840000,2.920000,2.800000,3.100000,3.01000,3.320000,3.140000,3.140000,3.090000
2,Canada,3.391500,3.407727,3.075714,2.944737,3.186364,3.278500,3.161500,3.288636,2.93500,3.037143,3.149167,3.221429,3.312381
3,Denmark,2.518000,2.448000,2.199000,2.114000,2.104000,2.069000,1.912000,2.213000,2.16600,2.454000,2.311000,2.314000,2.337000
4,France,3.150000,3.140000,2.940000,2.900000,2.990000,3.090000,3.010000,3.320000,3.15000,3.430000,3.260000,3.260000,3.240000
5,Germany,2.484500,2.458261,2.207727,2.167143,2.229565,2.305714,2.178889,2.483636,2.40550,2.741429,2.510500,2.562857,2.517000
6,Greece,3.650000,3.500000,3.310000,3.190000,3.160000,3.190000,3.050000,3.350000,3.29000,3.600000,3.440000,3.370000,NaN
7,India,7.020000,7.010000,6.910000,6.830000,6.790000,6.820000,6.780000,6.760000,6.73000,6.680000,6.460000,6.270000,NaN
8,Ireland,2.928700,2.858700,2.637000,2.545700,2.588800,2.626200,2.490600,2.789000,2.71640,3.048700,2.898100,2.887700,2.850200
9,Italy,3.942000,3.830000,3.682000,3.565000,3.501000,3.569000,3.321000,3.676000,3.57500,3.887000,3.712000,3.629000,3.501000


In [73]:
final_df = pd.merge(merged_df, yields_df, on = 'Country Name', how = 'left')
final_df.head()

,Country Name,Credit Rating,E Score,S Score,G Score,ESG Score,2024-06-01 00:00:00,2024-07-01 00:00:00,2024-08-01 00:00:00,2024-09-01 00:00:00,2024-10-01 00:00:00,2024-11-01 00:00:00,2024-12-01 00:00:00,2025-01-01 00:00:00,2025-02-01 00:00:00,2025-03-01 00:00:00,2025-04-01 00:00:00,2025-05-01 00:00:00,2025-06-01 00:00:00
0,Australia,100,0.547563,0.866146,0.821755,2.235464,4.2420,4.326000,3.976000,3.922000,4.267000,4.5440,4.3130,4.481000,4.423,4.421000,4.267000,4.350000,4.220000
1,Belgium,87,0.662349,0.773847,0.647842,2.084038,3.1200,3.060000,2.830000,2.790000,2.840000,2.9200,2.8000,3.100000,3.010,3.320000,3.140000,3.140000,3.090000
2,Canada,100,0.484865,0.779746,0.752887,2.017498,3.3915,3.407727,3.075714,2.944737,3.186364,3.2785,3.1615,3.288636,2.935,3.037143,3.149167,3.221429,3.312381
3,Denmark,100,0.750027,0.803905,0.846567,2.400499,2.5180,2.448000,2.199000,2.114000,2.104000,2.0690,1.9120,2.213000,2.166,2.454000,2.311000,2.314000,2.337000
4,France,88,0.797240,0.764189,0.625450,2.186880,3.1500,3.140000,2.940000,2.900000,2.990000,3.0900,3.0100,3.320000,3.150,3.430000,3.260000,3.260000,3.240000


In [74]:
bonds_df.to_csv('/Users/jamiewong/Documents/esg-financial-assistant/data/yields_cleaned.csv')